# Reaction Property Packages in IDAES

Introductory stuff

Note on building up classes using methods

## Reaction Parameter Block

More text

In [1]:
# Import Pyomo libraries
from pyomo.environ import (Constraint,
                           exp,
                           log,
                           Param,
                           Set,
                           units,
                           Var)

# Import IDAES cores
from idaes.core import (declare_process_block_class,
                        MaterialFlowBasis,
                        ReactionParameterBlock,
                        ReactionBlockDataBase,
                        ReactionBlockBase)
from idaes.core.util.constants import Constants as const
import idaes.logger as idaeslog

In [2]:
def define_kinetic_reactions(self):
    # Rate Reaction Index
    self.rate_reaction_idx = Set(initialize=["R1"])

    # Rate Reaction Stoichiometry
    self.rate_reaction_stoichiometry = {("R1", "Vap", "benzene"): 1,
                                        ("R1", "Vap", "toluene"): -1,
                                        ("R1", "Vap", "hydrogen"): -1,
                                        ("R1", "Vap", "methane"): 1,
                                        ("R1", "Vap", "diphenyl"): 0}

In [3]:
def define_equilibrium_reactions(self):
    # Equilibrium Reaction Index
    self.equilibrium_reaction_idx = Set(initialize=["E1"])

    # Equilibrium Reaction Stoichiometry
    self.equilibrium_reaction_stoichiometry = {
        ("E1", "Vap", "benzene"): -2,
        ("E1", "Vap", "toluene"): 0,
        ("E1", "Vap", "hydrogen"): 1,
        ("E1", "Vap", "methane"): 0,
        ("E1", "Vap", "diphenyl"): 1}

In [4]:
def define_parameters(self):
    # Arrhenius Constant
    self.arrhenius = Param(default=1.25e-9,
                           doc="Arrhenius constant",
                           units=units.mol/units.m**3/units.s/units.Pa**2)

    # Activation Energy
    self.energy_activation = Param(default=3800,
                                   doc="Activation energy [J/mol]",
                                   units=units.J/units.mol)

In [5]:
@declare_process_block_class("HDAReactionParameterBlock")
class HDAReactionParameterData(ReactionParameterBlock):
    """
    Reaction Parameter Block Class
    """

    def build(self):
        '''
        Callable method for Block construction.
        '''
        super(HDAReactionParameterData, self).build()

        self._reaction_block_class = HDAReactionBlock
        
        define_kinetic_reactions(self)
        define_equilibrium_reactions(self)
        define_parameters(self)

    @classmethod
    def define_metadata(cls, obj):
        obj.add_properties({
                'k_rxn': {'method': None},
                'reaction_rate': {'method': None}
                })
        obj.add_default_units({'time': units.s,
                               'length': units.m,
                               'mass': units.kg,
                               'amount': units.mol,
                               'temperature': units.K})

## Reaction Blocks

Even more text

In [6]:
class _HDAReactionBlock(ReactionBlockBase):

    def initialize(blk, outlvl=idaeslog.NOTSET, **kwargs):
        init_log = idaeslog.getInitLogger(blk.name, outlvl, tag="properties")
        init_log.info('Initialization Complete.')

In [7]:
def define_variables_and_parameters(self):
    self.k_rxn = Var(initialize=7e-10,
                    doc="Rate constant",
                    units=units.mol/units.m**3/units.s/units.Pa**2)

    self.k_eq = Param(initialize=10000,
                      doc="Equlibrium constant",
                      units=units.Pa)

    self.reaction_rate = Var(self.params.rate_reaction_idx,
                             initialize=0,
                             doc="Rate of reaction",
                             units=units.mol/units.m**3/units.s)

In [8]:
def define_rate_expression(self):
    self.arrhenius_equation = Constraint(
            expr=self.k_rxn == self.params.arrhenius * exp(
                -self.params.energy_activation /
                (const.gas_constant*self.state_ref.temperature)))

    def rate_rule(b, r):
        return b.reaction_rate[r] == (
                    b.k_rxn *
                    b.state_ref.mole_frac_comp["toluene"] *
                    b.state_ref.mole_frac_comp["hydrogen"] *
                    b.state_ref.pressure**2)
    self.rate_expression = Constraint(self.params.rate_reaction_idx,
                                      rule=rate_rule)

In [9]:
def define_equilibrium_expression(self):
    self.equilibrium_constraint = Constraint(
        expr=self.k_eq *
        self.state_ref.mole_frac_comp["benzene"] *
        self.state_ref.pressure ==
        self.state_ref.mole_frac_comp["diphenyl"] *
        self.state_ref.mole_frac_comp["hydrogen"] *
        self.state_ref.pressure**2)

In [10]:
@declare_process_block_class("HDAReactionBlock",
                             block_class=_HDAReactionBlock)
class HDAReactionBlockData(ReactionBlockDataBase):
    def build(self):

        super(HDAReactionBlockData, self).build()
        
        define_variables_and_parameters(self)
        define_rate_expression(self)
        define_equilibrium_expression(self)
    
    def get_reaction_rate_basis(b):
        return MaterialFlowBasis.molar

# Demonstration

In [17]:
from pyomo.environ import ConcreteModel, SolverFactory
from pyomo.util.check_units import assert_units_consistent

from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import CSTR

from thermophysical_property_example import HDAParameterBlock

from idaes.core.util.model_statistics import degrees_of_freedom

In [12]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.thermo_params = HDAParameterBlock()
m.fs.reaction_params = HDAReactionParameterBlock(
    default={"property_package": m.fs.thermo_params})

m.fs.reactor = CSTR(default={
    "property_package": m.fs.thermo_params,
    "reaction_package": m.fs.reaction_params,
    "has_equilibrium_reactions": True})

In [13]:
print("Degrees of Freedom: ", degrees_of_freedom(m))

Degrees of Freedom:  9


In [14]:
m.fs.reactor.inlet.flow_mol.fix(100)
m.fs.reactor.inlet.temperature.fix(500)
m.fs.reactor.inlet.pressure.fix(350000)
m.fs.reactor.inlet.mole_frac_comp[0, "benzene"].fix(0.1)
m.fs.reactor.inlet.mole_frac_comp[0, "toluene"].fix(0.4)
m.fs.reactor.inlet.mole_frac_comp[0, "hydrogen"].fix(0.4)
m.fs.reactor.inlet.mole_frac_comp[0, "methane"].fix(0.1)
m.fs.reactor.inlet.mole_frac_comp[0, "diphenyl"].fix(0.0)

m.fs.reactor.volume.fix(1)

print("Degrees of Freedom: ", degrees_of_freedom(m))

Degrees of Freedom:  0


In [18]:
m.fs.reactor.initialize(state_args={
    "flow_mol": 100,
    "mole_frac_comp": {
        "benzene": 0.15,
        "toluene": 0.35,
        "hydrogen": 0.35,
        "methane": 0.15,
        "diphenyl": 0.01},
    "temperature": 600,
    "pressure": 350000})

solver = SolverFactory('ipopt')
solver.solve(m, tee=True)

2020-11-06 14:11:37 [INFO] idaes.init.fs.reactor.control_volume.properties_in: Properties Initialized optimal - Optimal Solution Found.
2020-11-06 14:11:37 [INFO] idaes.init.fs.reactor.control_volume.properties_out: Properties Initialized optimal - Optimal Solution Found.
2020-11-06 14:11:37 [INFO] idaes.init.fs.reactor.control_volume.reactions: Initialization Complete.
2020-11-06 14:11:37 [INFO] idaes.init.fs.reactor.control_volume: Initialization Complete
2020-11-06 14:11:37 [INFO] idaes.init.fs.reactor.control_volume.properties_in: State Released.
2020-11-06 14:11:37 [INFO] idaes.init.fs.reactor: Initialization Complete: optimal - Optimal Solution Found
Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This versio

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 24, 'Number of variables': 24, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.08358550071716309}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [19]:
m.fs.reactor.outlet.display()

outlet : Size=1
    Key  : Name           : Value
    None :       flow_mol : {0.0: 100.0}
         : mole_frac_comp : {(0.0, 'benzene'): 0.1596262067323543, (0.0, 'diphenyl'): 0.013972841497705371, (0.0, 'hydrogen'): 0.32640095176994033, (0.0, 'methane'): 0.18757188972776503, (0.0, 'toluene'): 0.31242811027223494}
         :       pressure : {0.0: 350000.0}
         :    temperature : {0.0: 790.2120627373447}


In [20]:
assert_units_consistent(m)